In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import numpy as np
from logistic_regression_yt import LogisticRegressioYT
from sklearn.model_selection import train_test_split
from preprocess import Preprocess

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split


# Import necessary libraries
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


Loading data

In [2]:
raw_train_data = "../data/train.csv"
raw_test_data = "../data/test.csv"

Preprocessing

In [3]:
preproc = Preprocess()
raw_data = preproc.load_data(raw_train_data)
train_df, train_data = preproc.preprocess_data(raw_data)
train_data.shape

(44760, 20)

In [4]:
train_df.Label.value_counts(normalize=True)

0    0.785947
2    0.173280
1    0.040773
Name: Label, dtype: float64

In [5]:
X_train, y_train, X_valid, y_valid = preproc.train_valid_split(
    train_data, test_size=0.33, random_state=42)

In [6]:
X_train = preproc.normalize_data(X_train)
X_valid = preproc.normalize_data(X_valid)

Teste com uma classe só

In [7]:
y_train = (y_train == 1).astype(int)
y_valid = (y_valid == 1).astype(int)

In [8]:
y_valid

array([0, 0, 0, ..., 0, 0, 0])

In [10]:
# Standardize the features (important for many linear models)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_valid)

In [11]:
# Train the logistic regression model
clf = LogisticRegression(
    penalty='l2', 
    dual=False,
    tol=0.0001,
    C=1.0,
    fit_intercept=True,
    intercept_scaling=1,
    class_weight=None,
    random_state=None,
    solver='lbfgs',
    max_iter=100,
    multi_class='auto',
    verbose=0,
    warm_start=False,
    n_jobs=None,
    l1_ratio=None)

clf.fit(X_train, y_train)

/Users/rafaelapinter/miniconda3/envs/dc1-env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [12]:
# Predict on test data
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_valid, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 96.80%


In [13]:
def confusion_matrix(y_true, y_pred):
    """
    Compute confusion matrix for given true and predicted labels.

    :param y_true: Actual class labels
    :param y_pred: Predicted class labels
    :return: Confusion matrix
    """

    # Get unique class labels
    classes = np.unique(np.concatenate((y_true, y_pred)))
    num_classes = len(classes)

    # Create a mapping from class label to index
    label_to_index = {label: idx for idx, label in enumerate(classes)}

    # Initialize confusion matrix with zeros
    matrix = np.zeros((num_classes, num_classes), dtype=int)

    # Populate the confusion matrix
    for true_label, pred_label in zip(y_true, y_pred):
        matrix[label_to_index[true_label], label_to_index[pred_label]] += 1

    print("Confusion Matrix:")
    print(matrix, "\n")
    return matrix

In [15]:
confusion_matrix(y_valid, y_pred)

Confusion Matrix:
[[14120    73]
 [  400   178]] 



array([[14120,    73],
       [  400,   178]])